In [32]:
import os 

os.listdir('data')

['average_quantity.csv', 'cohort_counts.csv', 'online.csv']

In [58]:
import pandas as pd 
import datetime as dt

online = pd.read_csv('data/online.csv')
online.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom


# RFM Segmentation

세 개 지표(metric)에 기반하여 행동적(behavioral)으로 고객을 분류
- Recency (R)
- Frequency (F)
- Monetary (M) 

각각의 지표(metric)의 정의는 아래와 같음. 

- Recency - days since last customer transaction
- Frequency - number of transactions in the last 12 months
- Monetary Value - total spend in the last 12 months

메트릭은 다양한 방식으로 그룹핑될 수 있음. 
- Percentiles (e.g., quantiles)
- Pareto 80/20 cut 
- Custom - based on business knowledge

이번에는 Percentiles를 이용하여 분석. 프로세스는 아래와 같음

1. 측정한 메트릭에 맞게 고객들을 정렬함.(Sort customers based on that metric)
2. 동일한 크기의 그룹으로 (그룹 개수는 사전에 정의해놓음) 고객을 분류 (Break customers into a pre-defined number of groups of equal size)
3. 각 그룹에 레이블 배정 (Assign a label to each group)

## RFM 분석을 위한 Online 데이터셋 전처리

In [59]:
online['TotalSum'] = online['Quantity'] * online['UnitPrice']
online['InvoiceDate'] = pd.to_datetime(online['InvoiceDate'])
online.head()

,Unnamed: 0,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalSum
0,416792,572558,22745,POPPY'S PLAYHOUSE BEDROOM,6,2011-10-25 08:26:00,2.10,14286,United Kingdom,12.60
1,482904,577485,23196,VINTAGE LEAF MAGNETIC NOTEPAD,1,2011-11-20 11:56:00,1.45,16360,United Kingdom,1.45
2,263743,560034,23299,FOOD COVER WITH BEADS SET 2,6,2011-07-14 13:35:00,3.75,13933,United Kingdom,22.50
3,495549,578307,72349B,SET/6 PURPLE BUTTERFLY T-LIGHTS,1,2011-11-23 15:53:00,2.10,17290,United Kingdom,2.10
4,204384,554656,21756,BATH BUILDING BLOCK WORD,3,2011-05-25 13:36:00,5.95,17663,United Kingdom,17.85


송장 날짜 중 최솟값과 최댓값을 구하고, (비록 예전 데이터셋이지만...) 최근 데이터셋이라고 가정하고 Recency를 계산하기 위해 hypothetical한 snapshot date를 설정함

In [60]:
print('Min:{}; Max:{}'.format(min(online.InvoiceDate), max(online.InvoiceDate)))

Min:2010-12-01 08:26:00; Max:2011-12-09 12:49:00


In [61]:
snapshot_date = max(online.InvoiceDate) + dt.timedelta(days=1)

In [62]:
datamart = online.groupby(['CustomerID']).agg({
    #Recency 계산, 기준일이 되는 날짜로부터 고객의 가장 최근 거래가 얼마나 지났는지 계산
'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    # Frequency 계산, 송장 번호의 개수 카운트
'InvoiceNo': 'count',
    # TotalSum 값을 모두 합합
'TotalSum': 'sum'})

datamart

,InvoiceDate,InvoiceNo,TotalSum
CustomerID,,,
12747,2,27,992.82
12748,1,967,7522.06
12749,4,37,813.45
12820,3,17,268.02
12822,71,9,146.15
...,...,...,...
18280,278,2,38.70
18281,181,2,31.80
18282,8,2,30.70


In [63]:
datamart.rename(columns = {'InvoiceDate': 'Recency'
,'InvoiceNo': 'Frequency'
,'TotalSum': 'MonetaryValue'}, inplace=True)

datamart.head()

,Recency,Frequency,MonetaryValue
CustomerID,,,
12747,2,27,992.82
12748,1,967,7522.06
12749,4,37,813.45
12820,3,17,268.02
12822,71,9,146.15


## Recency Quartiles 

Recency는 값이 크다고 좋은 것이 아니고, 오히려 값이 작을 수록 지표가 좋은 것임. 그래서 레이블링을 역방향으로 할 필요가 있음. 

In [64]:
r_labels = range(4, 0, -1)
r_quartiles = pd.qcut(datamart['Recency'], 4, labels = r_labels)
datamart = datamart.assign(R = r_quartiles.values)
datamart

,Recency,Frequency,MonetaryValue,R
CustomerID,,,,
12747,2,27,992.82,4
12748,1,967,7522.06,4
12749,4,37,813.45,4
12820,3,17,268.02,4
12822,71,9,146.15,2
...,...,...,...,...
18280,278,2,38.70,1
18281,181,2,31.80,1
18282,8,2,30.70,4


## Frequency, Monetary Quartiles

In [65]:
f_labels = range(1,5)
m_labels = range(1,5)
f_quartiles = pd.qcut(datamart['Frequency'], 4, labels = f_labels)
m_quartiles = pd.qcut(datamart['MonetaryValue'], 4, labels = m_labels)
datamart = datamart.assign(F = f_quartiles.values)
datamart = datamart.assign(M = m_quartiles.values)
datamart

,Recency,Frequency,MonetaryValue,R,F,M
CustomerID,,,,,,
12747,2,27,992.82,4,4,4
12748,1,967,7522.06,4,4,4
12749,4,37,813.45,4,4,4
12820,3,17,268.02,4,3,3
12822,71,9,146.15,2,2,3
...,...,...,...,...,...,...
18280,278,2,38.70,1,1,1
18281,181,2,31.80,1,1,1
18282,8,2,30.70,4,1,1


## RFM 세그먼트, RFM 점수 구축

In [84]:
#def join_rfm(x): 
    #rfm quartile로 각각 그룹핑한 걸 합침 
    #return str(x['R']) + str(x['F']) + str(x['M'])

# datamart['RFM_Segment'] = datamart.apply(join_rfm, axis=1) 
#-> dlrjs wkRn 4.04.04 이런식으로 RFM Segment가 합쳐짐... 

for i in range(len(datamart['R'])): 
    datamart['RFM_Segment'].iloc[i] = str(datamart['R'].iloc[i]) + str(datamart['F'].iloc[i]) + str(datamart['M'].iloc[i])

datamart['RFM_Score'] = datamart[['R','F','M']].sum(axis=1)

In [91]:
datamart

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12747,2,27,992.82,4,4,4,444,12
12748,1,967,7522.06,4,4,4,444,12
12749,4,37,813.45,4,4,4,444,12
12820,3,17,268.02,4,3,3,433,10
12822,71,9,146.15,2,2,3,223,7
...,...,...,...,...,...,...,...,...
18280,278,2,38.70,1,1,1,111,3
18281,181,2,31.80,1,1,1,111,3
18282,8,2,30.70,4,1,1,411,6


## RFM 세그먼트 분석

In [92]:
datamart.groupby('RFM_Segment').size().sort_values(ascending=False)[:10]

RFM_Segment
444    382
111    346
211    168
344    162
233    132
222    130
311    120
433    119
333    118
122    116
dtype: int64

In [93]:
#최하위 세그먼트
datamart[datamart['RFM_Segment']=='111'][:5]

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12837,173,2,10.55,1,1,1,111,3
12852,295,2,32.55,1,1,1,111,3
12902,265,4,42.03,1,1,1,111,3
12922,161,4,57.24,1,1,1,111,3
12929,312,3,42.90,1,1,1,111,3


In [94]:
datamart[datamart['RFM_Segment']=='444'][:5]

,Recency,Frequency,MonetaryValue,R,F,M,RFM_Segment,RFM_Score
CustomerID,,,,,,,,
12747,2,27,992.82,4,4,4,444,12
12748,1,967,7522.06,4,4,4,444,12
12749,4,37,813.45,4,4,4,444,12
12839,3,59,1048.93,4,4,4,444,12
12841,5,78,713.65,4,4,4,444,12


In [95]:
datamart.groupby('RFM_Score').agg({'Recency': 'mean', 
                                   'Frequency': 'mean',
                                   'MonetaryValue': ['mean','count'] }).round(1)

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Score                                      
3           256.7       2.0          28.5   346
4           175.7       3.2          48.0   348
5           145.3       4.3          79.6   406
6           105.4       6.4         149.4   433
7            82.8       8.9         163.1   384
8            63.4      12.8         198.6   382
9            48.4      16.8         330.0   347
10           32.9      24.5         438.8   364
11           21.6      38.8         709.4   309
12            7.6      77.3        1709.0   382

In [96]:
datamart.groupby('RFM_Score').agg({'Recency': 'mean',
                                   'Frequency': 'mean',
                                   'MonetaryValue': ['mean','count'] }).round(1)

Recency Frequency MonetaryValue      
             mean      mean          mean count
RFM_Score                                      
3           256.7       2.0          28.5   346
4           175.7       3.2          48.0   348
5           145.3       4.3          79.6   406
6           105.4       6.4         149.4   433
7            82.8       8.9         163.1   384
8            63.4      12.8         198.6   382
9            48.4      16.8         330.0   347
10           32.9      24.5         438.8   364
11           21.6      38.8         709.4   309
12            7.6      77.3        1709.0   382

In [99]:
def segment_me(df):
    if df['RFM_Score'] >= 9: 
        return 'Gold'
    elif (df['RFM_Score'] >= 5) and (df['RFM_Score'] < 9): 
        return 'Silver'
    else: 
        return 'Bronze'
    
datamart['General_Segment'] = datamart.apply(segment_me, axis=1)
datamart.groupby('General_Segment').agg({'Recency': 'mean',
                                        'Frequency': 'mean',
                                        'MonetaryValue': ['mean','count']}).round(1)

Recency Frequency MonetaryValue      
                   mean      mean          mean count
General_Segment                                      
Bronze            216.1       2.6          38.3   694
Gold               27.4      40.1         817.6  1402
Silver            100.1       8.0         146.7  1605